In [1]:
import pandas as pd
import numpy as np

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, mean_absolute_error as MAE
from sklearn.pipeline import make_pipeline

import matplotlib.pyplot as plt
%matplotlib inline

### Load data and preprocess

In [2]:
df = pd.read_csv('data/VCI_info_block1.txt', delim_whitespace=True)  # read the csv file and get all the (n,n') and associated matrix elements (lower triangle)
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy())                    # all matrix elements into a 1D array
wH_mat = np.zeros((161,161))                                                   # initialize the H-matrix
ind = np.tril_indices(161)                                                     # get the indices of the lower-triangular H-matrix
wH_mat[ind] = wmat_array                                                       # fill in the lower triangle of the H-matrix using the 1D array of the matrix elements
wH_diag = np.diag(wH_mat)                                                      # get the diagonal entries (we always use the exact values for the diagonal terms)

In [3]:
thresh = 0.00001                                                               # set a threshold
df = df.assign(label = (abs(df[["<i1j1k1|H|i2j2k2>"]]) > thresh)*1 )             # assign 0/1 to each matrix element
df

,i1,j1,k1,i2,j2,k2,<i1j1k1|H|i2j2k2>,label
0,0,0,0,0,0,0,2.121571e-02,1
1,0,0,0,1,0,0,1.000000e-08,0
2,1,0,0,1,0,0,2.851704e-02,1
3,0,0,0,2,0,0,7.000000e-08,0
4,1,0,0,2,0,0,5.000000e-08,0
...,...,...,...,...,...,...,...,...
13036,3,1,6,1,1,8,-3.171950e-03,1
13037,1,2,6,1,1,8,-1.432844e-02,1
13038,2,2,6,1,1,8,-7.816000e-04,1
13039,1,3,6,1,1,8,-4.898860e-03,1


In [4]:
Xo = df[["i1", "j1", "k1", "i2", "j2", "k2"]].to_numpy(dtype=float)    # (n, n') as the Xo
X = np.copy(Xo)                                                        # (n, n-n') as the X, which will be used as the features for ML models
X[:,0:3] = Xo[:,0:3]
X[:,3:6] = Xo[:,3:6] - Xo[:,0:3]
y = np.ravel(df["label"].to_numpy())                                   # labels
print(X.shape, y.shape)

print(sum(y))
print((13041. - sum(y)) / 130.41)                                      # compute the percentage of 0's in the matrix elements

(13041, 6) (13041,)
5730
56.061651713825626


In [5]:
# The exact H-matrix and the eigenvalues/eigenvectors
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy())
wH_mat = np.zeros((161,161))
ind = np.tril_indices(161)
wH_mat[ind] = wmat_array
wH_mat = wH_mat + wH_mat.T - 2*np.diag(np.diag(wH_mat)) + np.diag(wH_diag)

w, v = np.linalg.eig(wH_mat)
freq1 = np.sort(w) * 219474.63
print(freq1[1:21]-freq1[0])

[ 1594.37629943  3150.86730055  3656.11784269  4665.93113308
  5233.74613343  6134.59243619  6773.57597478  7199.97647588
  7443.48506555  7542.12258445  8269.78565885  8652.64486927
  8760.56953518  8998.7272941   9367.0028636   9762.84034428
 10285.04289498 10521.91238925 10602.90093816 10756.1075468 ]


In [6]:
# Approximate H-matrix if all elements with label 0 are set to 0
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy()) * y
wH_mat = np.zeros((161,161))
ind = np.tril_indices(161)
wH_mat[ind] = wmat_array
wH_mat = wH_mat + wH_mat.T - 2*np.diag(np.diag(wH_mat)) + np.diag(wH_diag)

w, v = np.linalg.eig(wH_mat)
freq2 = np.sort(w) * 219474.63
print(freq2[1:21]-freq2[0])
print("MAE:", MAE(freq1, freq2))

[ 1594.3670419   3150.82742649  3656.14319279  4665.99489192
  5233.70519508  6134.68417322  6773.43359898  7200.08772266
  7443.48968142  7542.28056641  8269.52026605  8652.63369973
  8760.44391271  8998.84485117  9366.91373024  9762.99283685
 10285.26246398 10521.70776892 10603.17009123 10755.93271344]
MAE: 0.2747757351678911


In [7]:
# random train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.6)
print(X_train.shape, X_test.shape)

(5216, 6) (7825, 6)


### Random Forest

In [8]:
# Train a random-forest classifier
rf = RandomForestClassifier(n_estimators=20, max_depth=10)
rf.fit(X_train, y_train)

print("Training accuracy:", rf.score(X_train, y_train))
print("Testing accuracy:", rf.score(X_test, y_test))

Training accuracy: 0.9616564417177914
Testing accuracy: 0.9300958466453674


In [9]:
# Approximate H-matrix if using ML-predicted labels and setting elements with label 0 as 0.
y_pred = rf.predict(X)
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy()) * y_pred
wH_mat = np.zeros((161,161))
ind = np.tril_indices(161)
wH_mat[ind] = wmat_array
wH_mat = wH_mat + wH_mat.T - 2*np.diag(np.diag(wH_mat)) + np.diag(wH_diag)

w, v = np.linalg.eig(wH_mat)
freq3 = np.sort(w) * 219474.63
print(freq3[1:21]-freq3[0])
print(MAE(freq1, freq3))

[ 1594.33015279  3150.8457598   3656.40663636  4666.02327861
  5233.61683213  6134.74763827  6773.48034497  7201.95363152
  7441.55971946  7542.23799715  8269.34703232  8652.5279939
  8760.43962361  8998.6889899   9366.65795515  9763.00079134
 10285.15691452 10521.60854198 10605.46911192 10755.82752216]
0.8051803561310187


### MLP Classifier

In [10]:
# Train a multi-layer perceptron classifier
mlp = make_pipeline(StandardScaler(), MLPClassifier(hidden_layer_sizes=(15, 15), early_stopping=True, activation='relu', validation_fraction=0.2))
mlp.fit(X_train, y_train)

print("Training accuracy:", mlp.score(X_train, y_train))
print("Testing accuracy:", mlp.score(X_test, y_test))

Training accuracy: 0.8870782208588958
Testing accuracy: 0.8792332268370607


In [11]:
# Approximate H-matrix if using ML-predicted labels and setting elements with label 0 as 0
y_pred = mlp.predict(X)
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy()) * y_pred
wH_mat = np.zeros((161,161))
ind = np.tril_indices(161)
wH_mat[ind] = wmat_array
wH_mat = wH_mat + wH_mat.T - 2*np.diag(np.diag(wH_mat)) + np.diag(wH_diag)

w, v = np.linalg.eig(wH_mat)
freq4 = np.sort(w) * 219474.63
print(freq4[1:21]-freq4[0])
print(MAE(freq1, freq4))

[ 1594.46866981  3150.8037854   3656.45357047  4665.42495913
  5234.15029111  6133.95786536  6774.37392646  7201.98341267
  7441.43774658  7542.35538316  8270.15921681  8653.00635862
  8761.86640664  8996.97175561  9365.17207613  9763.86088448
 10286.79856719 10520.49210849 10600.33895855 10755.69381461]
1.1688555498084163


### SVM Classifier

In [12]:
# Train a SVM classifier
svm = make_pipeline(StandardScaler(), svm.SVC(kernel='rbf'))
svm.fit(X_train, y_train)

print("Training accuracy:", svm.score(X_train, y_train))
print("Testing accuracy:", svm.score(X_test, y_test))

Training accuracy: 0.8953220858895705
Testing accuracy: 0.8805111821086262


In [13]:
# Approximate H-matrix if using ML-predicted labels and setting elements with label 0 as 0.
y_pred = svm.predict(X)
wmat_array = np.ravel(df[["<i1j1k1|H|i2j2k2>"]].to_numpy()) * y_pred
wH_mat = np.zeros((161,161))
ind = np.tril_indices(161)
wH_mat[ind] = wmat_array
wH_mat = wH_mat + wH_mat.T - 2*np.diag(np.diag(wH_mat)) + np.diag(wH_diag)

w, v = np.linalg.eig(wH_mat)
freq5 = np.sort(w) * 219474.63
print(freq5[1:21]-freq5[0])
print(MAE(freq1, freq5))

[ 1594.42837107  3150.8751558   3656.4904489   4665.5847493
  5234.36803705  6134.04207919  6773.63187219  7201.93099598
  7441.48295349  7543.35196415  8269.47987208  8654.95056291
  8759.22802915  8998.5390412   9362.31429712  9767.35056527
 10284.02629437 10521.79923331 10605.10673552 10755.25189452]
1.2307984145600694
